[![xeus-clang-repl](images/xeus-clang-repl.png)](https://github.com/compiler-research/xeus-clang-repl)

A Jupyter kernel for C++ based on the `clang-repl` C++ interpreter and the `xeus` native implementation of the Jupyter protocol, xeus.

- GitHub repository: https://github.com/compiler-research/xeus-clang-repl
- Online documentation: https://xeus-clang-repl.readthedocs.io/

## Usage

<div style="background: #efffed;
            border: 1px solid grey;
            margin: 8px 0 8px 0;
            text-align: center;
            padding: 8px; ">
    <i class="fa-play fa" 
       style="font-size: 40px;
              line-height: 40px;
              margin: 8px;
              color: #444;">
    </i>
    <div>
    To run the selected code cell, hit <pre style="background: #efffed">Shift + Enter</pre>
    </div>
</div>

## Output and error streams

`std::cout` and `std::cerr` are redirected to the notebook frontend.

In [1]:
#include <iostream>

std::cout << "some output" << std::endl;

some output


In [2]:
std::cerr << "some error" << std::endl;

some error


In [3]:
#include <stdexcept>

In [4]:
throw std::runtime_error("Unknown exception");

Standard Exception: Unknown exception

Omitting the `;` in the last statement of a cell results in an output being printed

In [5]:
int j = 5;

In [6]:
//// Fix:
//j

# Interpreting the C++ programming language

`clang-repl` has a broad support of the features of C++. You can define functions, classes, templates, etc ...

## Functions

In [7]:
double sqr(double a)
{
    return a * a;
}

In [8]:
double a = 2.5;
double asqr = sqr(a);
//// Fix:
//asqr

## Classes

In [9]:
class Foo
{
public:

    virtual ~Foo() {}
    
    virtual void print(double value) const
    {
        std::cout << "Foo value = " << value << std::endl;
    }
};

In [10]:
Foo bar;
bar.print(1.2);

Foo value = 1.2


## Polymorphism

In [11]:
class Bar : public Foo
{
public:

    virtual ~Bar() {}
    
    virtual void print(double value) const
    {
        std::cout << "Bar value = " << 2 * value << std::endl;
    }
};

In [12]:
Foo* bar2 = new Bar;
bar2->print(1.2);
delete bar2;

Bar value = 2.4


## Templates

In [13]:
#include <typeinfo>

template <class T>
class FooT
{
public:
    
    explicit FooT(const T& t) : m_t(t) {}
    
    void print() const
    {
        std::cout << typeid(T).name() << " m_t = " << m_t << std::endl;
    }
    
private:
    
    T m_t;
};

In [14]:
FooT<double> foot(1.2);
foot.print();

d m_t = 1.2


## C++11 / C++14 support

In [15]:
class Foo11
{
public:
    
    Foo11() { std::cout << "Foo11 default constructor" << std::endl; }
    Foo11(const Foo11&) { std::cout << "Foo11 copy constructor" << std::endl; }
    Foo11(Foo11&&) { std::cout << "Foo11 move constructor" << std::endl; }
};

In [16]:
Foo11 f1;
Foo11 f2(f1);
Foo11 f3(std::move(f1));

Foo11 default constructor
Foo11 copy constructor
Foo11 move constructor


In [17]:
#include <vector>

std::vector<int> v = { 1, 2, 3 };
auto iter = ++v.begin();
//// Fix:
//v

In [18]:
//// Fix:
//*iter

... and also lambda, universal references, `decltype`, etc ...

## Documentation and completion

 - Documentation for types of the standard library is retrieved on cppreference.com.
 - The quick-help feature can also be enabled for user-defined types and third-party libraries. More documentation on this feature is available at https://xeus-clang-repl.readthedocs.io/en/latest/inline_help.html.


In [19]:
//// Fix:
//?std::vector

## Using the `display_data` mechanism

For a user-defined type `T`, the rich rendering in the notebook and JupyterLab can be enabled by by implementing the function `nl::json mime_bundle_repr(const T& im)`, which returns the JSON mime bundle for that type.

More documentation on the rich display system of Jupyter and Xeus-clang-repl is available at https://xeus-clang-repl.readthedocs.io/en/latest/rich_display.html

### Image example

In [20]:
#include <string>
#include <fstream>

#include "nlohmann/json.hpp"

#include "xtl/xbase64.hpp"

namespace nl = nlohmann;

namespace im
{
    struct image
    {   
        inline image(const std::string& filename)
        {
            std::ifstream fin(filename, std::ios::binary);   
            m_buffer << fin.rdbuf();
        }
        
        std::stringstream m_buffer;
    };
    
    nl::json mime_bundle_repr(const image& i)
    {
        auto bundle = nl::json::object();
        bundle["image/png"] = xtl::base64encode(i.m_buffer.str());
        return bundle;
    }
}

In [21]:
im::image marie("images/marie.png");
//// Fix:
//marie

### Audio example

In [22]:
#include <string>
#include <fstream>

#include "nlohmann/json.hpp"

#include "xtl/xbase64.hpp"

namespace nl = nlohmann;

namespace au
{
    struct audio
    {   
        inline audio(const std::string& filename)
        {
            std::ifstream fin(filename, std::ios::binary);   
            m_buffer << fin.rdbuf();
        }
        
        std::stringstream m_buffer;
    };
    
    nl::json mime_bundle_repr(const audio& a)
    {
        auto bundle = nl::json::object();
        bundle["text/html"] =
           std::string("<audio controls=\"controls\"><source src=\"data:audio/wav;base64,")
           + xtl::base64encode(a.m_buffer.str()) +
            "\" type=\"audio/wav\" /></audio>";
        return bundle;
    }
}

In [23]:
au::audio drums("audio/audio.wav");
//// Fix:
//drums

### Display

In [24]:
//// Fix:
//#include "xcpp/xdisplay.hpp"

In [25]:
//// Fix:
//xcpp::display(drums);

### Update-display

In [26]:
#include <string>
//// Fix:
//#include "xcpp/xdisplay.hpp"

#include "nlohmann/json.hpp"

namespace nl = nlohmann;

namespace ht
{
    struct html
    {   
        inline html(const std::string& content)
        {
            m_content = content;
        }
        std::string m_content;
    };

    nl::json mime_bundle_repr(const html& a)
    {
        auto bundle = nl::json::object();
        bundle["text/html"] = a.m_content;
        return bundle;
    }
}

// A blue rectangle
//// Fix:
/*ht::html rect(R"(
<div style='
    width: 90px;
    height: 50px;
    line-height: 50px;
    background-color: blue;
    color: white;
    text-align: center;'>
Original
</div>)");*/

In [27]:
//// Fix:
//xcpp::display(rect, "some_display_id");

In [28]:
// Update the rectangle to be red
//// Fix:
/*rect.m_content = R"(
<div style='
    width: 90px;
    height: 50px;
    line-height: 50px;
    background-color: red;
    color: white;
    text-align: center;'>
Updated
</div>)";*/

//xcpp::display(rect, "some_display_id", true);

### Clear output

In [29]:
#include <chrono>
#include <iostream>
#include <thread>

//// Fix:
//#include "xcpp/xdisplay.hpp"

In [30]:
//// Fix:
/*std::cout << "hello" << std::endl;
std::this_thread::sleep_for(std::chrono::seconds(1));
xcpp::clear_output();  // will flicker when replacing "hello" with "goodbye"
std::this_thread::sleep_for(std::chrono::seconds(1));
std::cout << "goodbye" << std::endl;*/

In [31]:
//// Fix:
/*std::cout << "hello" << std::endl;
std::this_thread::sleep_for(std::chrono::seconds(1));
xcpp::clear_output(true);  // prevents flickering
std::this_thread::sleep_for(std::chrono::seconds(1));
std::cout << "goodbye" << std::endl;*/